<a href="https://colab.research.google.com/github/marcospiau/ia368-dd-dl4ir/blob/main/aula05-doc2query/aula05_doc2query_indexing_eval.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# This notebook will be used to index texts and eval results

# TL;DR: GO to `Calculating metrics for runs` for results

# Installs and imports

In [2]:
!free -mh
!lscpu

              total        used        free      shared  buff/cache   available
Mem:           35Gi       1.4Gi        15Gi        27Mi        18Gi        33Gi
Swap:            0B          0B          0B
Architecture:                    x86_64
CPU op-mode(s):                  32-bit, 64-bit
Byte Order:                      Little Endian
Address sizes:                   46 bits physical, 48 bits virtual
CPU(s):                          40
On-line CPU(s) list:             0-39
Thread(s) per core:              2
Core(s) per socket:              20
Socket(s):                       1
NUMA node(s):                    1
Vendor ID:                       GenuineIntel
CPU family:                      6
Model:                           63
Model name:                      Intel(R) Xeon(R) CPU @ 2.30GHz
Stepping:                        0
CPU MHz:                         2299.998
BogoMIPS:                        4599.99
Hypervisor vendor:               KVM
Virtualization type:             full
L1d c

In [2]:
%%capture
!wget -nc https://raw.githubusercontent.com/marcospiau/ia368-dd-dl4ir/main/scripts/install_anserini.sh && chmod +x install_anserini.sh && time ./install_anserini.sh

In [3]:
%%capture
!pip install -q ftfy polars toolz cytoolz transformers datasets gcsfs
!pip install -U t5[gcp,cache-tasks]==0.9.3
!pip install -U jaxlib
!sudo apt install -qq tree htop

**PS.: notebook must be restarted after packages above are installed.**

# gcloud authentication

In [1]:
import os
from google.colab import auth

os.environ['USE_AUTH_EPHEM'] = '0'
auth.authenticate_user(clear_output=False)

Go to the following link in your browser:

    https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=32555940559.apps.googleusercontent.com&redirect_uri=https%3A%2F%2Fsdk.cloud.google.com%2Fauthcode.html&scope=openid+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fappengine.admin+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fsqlservice.login+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcompute+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Faccounts.reauth+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&state=msHX7kONI3bjq4BrsaL7pXyGiEiE5m&prompt=consent&access_type=offline&code_challenge=E7fl2YjeyDQc3ugp4yfT6DuhLiOKSdxoY79zDTLa2u8&code_challenge_method=S256

Enter authorization code: 4/0AVHEtk4JVEfCsOwkaklFIzoZnSZ_TVNYIEPwQRc4zALMDX8mYEjqMqSaALyi3CzCeIjr8w


In [2]:
!gcloud auth login

Go to the following link in your browser:

    https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=32555940559.apps.googleusercontent.com&redirect_uri=https%3A%2F%2Fsdk.cloud.google.com%2Fauthcode.html&scope=openid+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fappengine.admin+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fsqlservice.login+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcompute+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Faccounts.reauth&state=G7FTZOkKiz1Pv7ZPcCe1fd7V1XYfDC&prompt=consent&access_type=offline&code_challenge=hNIeIdwvHwBbHeL5tY45anTFESGBY-fjuxryfjqEoNY&code_challenge_method=S256

Enter authorization code: 4/0AVHEtk6FHOXGWliHUOMPm8WT1IwZhTIbzOzsTIMcvgOZJ7B7DfRqyZXd116QRFOvQkjy-A

You are now logged in as [m136825@dac.unicamp.br].
Your current project is [None].  You can change this setting by running:
  $ gcloud config set project PROJECT_ID


# Preparing qrels and queries files

## RASCUNHO, TIRAR!!

In [67]:
import os
import datasets
import toolz
import multiprocessing as mp
import pandas as pd
from collections import Counter, defaultdict
import itertools
import polars as pl

import more_itertools
from etils import epath
from pathlib import Path
import functools
from tqdm import tqdm
import multiprocessing as mp


pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_colwidth', 1000)

os.environ['POLARS_FMT_STR_LEN']='1000'

In [ ]:
from transformers import T5Tokenizer

In [ ]:
tokenizer = T5Tokenizer.from_pretrained('t5-base')

/usr/local/lib/python3.9/dist-packages/transformers/models/t5/tokenization_t5.py:163: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


In [ ]:
def concat_title_and_text(ex):
    return {'title_and_text': f"{ex['title']}. {ex['text']}"}

def encode(batch):
    lengths = tokenizer(batch['title_and_text'], return_length=True)['length']
    return {'lengths': lengths}

In [ ]:
ds_corpus = datasets.load_dataset('BeIR/trec-covid', 'corpus')['corpus']
ds_corpus = ds_corpus.map(concat_title_and_text)

  0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
token_lengths = pd.Series(ds_corpus.map(encode, batched=True, num_proc=mp.cpu_count())['lengths'])

In [ ]:
tokenizer(ds_corpus.select(range(1))['title_and_text'])

{'input_ids': [[14067, 753, 13, 1543, 18, 1409, 1926, 499, 509, 21178, 9, 30195, 15, 13315, 44, 2671, 28508, 9, 702, 172, 636, 4457, 6, 1022, 26, 26, 9, 107, 6, 11279, 13849, 5, 3, 10539, 683, 14196, 8087, 10, 100, 29825, 5059, 1132, 8788, 8, 29969, 1863, 11, 3739, 753, 13, 1283, 1221, 28, 1543, 18, 1409, 1926, 499, 509, 21178, 9, 30195, 15, 13315, 44, 2671, 28508, 9, 702, 172, 636, 4457, 6, 1022, 26, 26, 9, 107, 6, 11279, 13849, 5, 3, 24506, 6299, 3592, 10, 18027, 28, 1465, 283, 5, 30195, 15, 9757, 45, 19944, 19622, 7, 45, 1762, 6622, 190, 1882, 6260, 130, 4313, 190, 8, 5893, 6420, 6427, 3187, 5, 15054, 7, 13, 1221, 130, 9112, 5, 3, 12200, 4254, 4578, 10, 1283, 1221, 130, 4313, 6, 5400, 13642, 15967, 6210, 13, 4068, 831, 7209, 5, 1377, 13315, 14156, 15967, 6210, 130, 573, 18, 9, 75, 1169, 1271, 5, 37, 7952, 4161, 66, 1246, 1637, 68, 47, 167, 1017, 16, 9806, 7, 6918, 15967, 6210, 11, 554, 18, 6646, 502, 4743, 15967, 6210, 5, 94, 6935, 215, 18, 7775, 68, 47, 167, 1017, 16, 8, 1590, 41, 

In [ ]:
token_lengths.describe().to_frame().round(2)

,0
count,171332.00
mean,266.93
std,254.57
min,3.00
25%,57.00
50%,268.00
75%,406.00
max,42655.00


In [ ]:
token_lengths.gt(512).value_counts().to_frame()

,0
False,154953
True,16379


In [4]:
ds_qrels = datasets.load_dataset('BeIR/trec-covid-qrels')['test']
ds_qrels

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating test split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/BeIR___csv/BeIR--trec-covid-qrels-1766e3af5b0b856a/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

Dataset({
    features: ['query-id', 'corpus-id', 'score'],
    num_rows: 66336
})

In [8]:
a = ds_qrels.to_pandas().apply(lambda x: (str(x['query-id']), str(x['corpus-id'])), axis=1)
a = sorted(a)
Counter(a).most_common(10)

[(('1', '005b2j4b'), 1),
 (('1', '00fmeepz'), 1),
 (('1', '0194oljo'), 1),
 (('1', '021q9884'), 1),
 (('1', '02f0opkr'), 1),
 (('1', '02say5f1'), 1),
 (('1', '0376d6vf'), 1),
 (('1', '047xpt2c'), 1),
 (('1', '04ftw7k9'), 1),
 (('1', '05vx82oo'), 1)]

In [9]:
b = pd.read_csv('tools/topics-and-qrels/qrels.beir-v1.0.0-trec-covid.test.txt', sep=' ', header=None).apply(lambda x: (str(x[0]), str(x[2])), axis=1)
b = sorted(b)


In [11]:
# There are two ids missing from qrels in anserini tools
print(set(a) - set(b))
print(set(b) - set(a))

{('50', 'svo94kuo'), ('38', '9hbib8b3')}
set()


## Convert qrels to TREC format

In [5]:
def convert_qrels_to_trec_format(output_file):
    def to_line(ex):
        # qid 0 docid score
        return '{} 0 {} {}\n'.format(ex['query-id'], ex['corpus-id'],
                                   ex['score'])
    ds = datasets.load_dataset('BeIR/trec-covid-qrels')['test']
    with open(output_file, 'w') as f:
        for item in ds.to_list():
            line = to_line(item)
            f.write(line)

QRELS_FILE = 'trec-covid-qrels_trec_format.txt'

convert_qrels_to_trec_format(QRELS_FILE)
!wc -l {QRELS_FILE}
!head {QRELS_FILE}

  0%|          | 0/1 [00:00<?, ?it/s]

66336 trec-covid-qrels_trec_format.txt
1 0 005b2j4b 2
1 0 00fmeepz 1
1 0 g7dhmyyo 2
1 0 0194oljo 1
1 0 021q9884 1
1 0 02f0opkr 1
1 0 047xpt2c 0
1 0 04ftw7k9 0
1 0 pl9ht0d0 0
1 0 05vx82oo 0


## Convert queries to TREC format

~Queries are already in `anserini/src/main/resources/topics-and-qrels/topics.beir-v1.0.0-trec-covid.test.tsv.gz`, so we are going to use it instead of downloading from HuggingFace.~



In [6]:
def convert_queries_to_trec_format(output_file):
    ds = datasets.load_dataset('BeIR/trec-covid', 'queries')['queries']
    ds.to_csv(output_file, header=None, sep='\t', columns=['_id', 'text'])

QUERIES_FILE = 'trec-covid-queries_trec_format.txt'

convert_queries_to_trec_format(QUERIES_FILE)
!wc -l {QUERIES_FILE}
!cat {QUERIES_FILE}

Generating queries split: 0 examples [00:00, ? examples/s]

Dataset trec-covid downloaded and prepared to /root/.cache/huggingface/datasets/BeIR___trec-covid/queries/0.0.0/093f1fe2ffa7a9c72fa48239c8f279b51d6b171abd77737c7fd1406125307599. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

Creating CSV from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

50 trec-covid-queries_trec_format.txt
1	what is the origin of COVID-19
2	how does the coronavirus respond to changes in the weather
3	will SARS-CoV2 infected people develop immunity? Is cross protection possible?
4	what causes death from Covid-19?
5	what drugs have been active against SARS-CoV or SARS-CoV-2 in animal studies?
6	what types of rapid testing for Covid-19 have been developed?
7	are there serological tests that detect antibodies to coronavirus?
8	how has lack of testing availability led to underreporting of true incidence of Covid-19?
9	how has COVID-19 affected Canada
10	has social distancing had an impact on slowing the spread of COVID-19?
11	what are the guidelines for triaging patients infected with coronavirus?
12	what are best practices in hospitals and at home in maintaining quarantine?
13	what are the transmission routes of coronavirus?
14	what evidence is there related to COVID-19 super spreaders
15	how long can the coronavirus live outside the body
16	how long doe

# Download pre-built index from pyserini

Following instructions, we will download the pre-built index `beir-v1.0.0-trec-covid-flat`.

In [7]:
from pyserini.index.lucene import IndexReader
from pyserini.search.lucene import LuceneSearcher
from pyserini.util import download_prebuilt_index

Just download file:

In [8]:
PREBUILT_INDEX_DIR = download_prebuilt_index('beir-v1.0.0-trec-covid.flat',
                                             verbose=True)
print(f'PREBUILT_INDEX_DIR is {PREBUILT_INDEX_DIR}')
!ls -lht {PREBUILT_INDEX_DIR}

lucene-index.beir-v1.0.0-trec-covid.flat.20221116.505594.tar.gz: 216MB [00:02, 98.1MB/s]                          


Extracting /root/.cache/pyserini/indexes/lucene-index.beir-v1.0.0-trec-covid.flat.20221116.505594.tar.gz into /root/.cache/pyserini/indexes/lucene-index.beir-v1.0.0-trec-covid.flat.20221116.505594.57b812594b11d064a23123137ae7dade...
PREBUILT_INDEX_DIR is /root/.cache/pyserini/indexes/lucene-index.beir-v1.0.0-trec-covid.flat.20221116.505594.57b812594b11d064a23123137ae7dade
total 258M
-rw-rw-r-- 1 2537 2603  340 Nov 16 15:21 _0.fnm
-rw-rw-r-- 1 2537 2603  19M Nov 16 15:21 _0_Lucene90_0.doc
-rw-rw-r-- 1 2537 2603  22M Nov 16 15:21 _0_Lucene90_0.pos
-rw-rw-r-- 1 2537 2603 3.6M Nov 16 15:21 _0_Lucene90_0.tim
-rw-rw-r-- 1 2537 2603  81K Nov 16 15:21 _0_Lucene90_0.tip
-rw-rw-r-- 1 2537 2603  306 Nov 16 15:21 _0_Lucene90_0.tmd
-rw-rw-r-- 1 2537 2603  516 Nov 16 15:21 _0.si
-rw-rw-r-- 1 2537 2603  154 Nov 16 15:21 segments_1
-rw-rw-r-- 1 2537 2603  242 Nov 16 15:21 _0.fdm
-rw-rw-r-- 1 2537 2603 127M Nov 16 15:21 _0.fdt
-rw-rw-r-- 1 2537 2603  12K Nov 16 15:21 _0.fdx
-rw-rw-r-- 1 2537 2603 1.4M 

Load index and show statistics:

In [9]:
index_reader = IndexReader.from_prebuilt_index('beir-v1.0.0-trec-covid.flat',
                                               verbose=True)
index_reader

Attempting to initialize pre-built index beir-v1.0.0-trec-covid.flat.
/root/.cache/pyserini/indexes/lucene-index.beir-v1.0.0-trec-covid.flat.20221116.505594.57b812594b11d064a23123137ae7dade already exists, skipping download.
Initializing beir-v1.0.0-trec-covid.flat...
{'total_terms': 20822821, 'documents': 171331, 'non_empty_documents': 171331, 'unique_terms': 202648}
Index passes consistency checks against pre-built index 'beir-v1.0.0-trec-covid.flat'!


# Validating results from anserini regressions on Beir TREC Covid

We will first attempt to replicate Anserini's results for TREC-COVID. Essentially, we will follow the instructions provided in [Anserini Regressions: BEIR (v1.0.0) — TREC-COVID](https://github.com/castorini/anserini/blob/master/docs/regressions-beir-v1.0.0-trec-covid-flat.md).

Another important file is this YAML, which contains information about evaluation and index building: https://github.com/castorini/anserini/blob/master/src/main/resources/regression/beir-v1.0.0-trec-covid-flat.yaml

In [29]:
import shlex
import subprocess

TREC_EVAL_BIN_PATH = './tools/eval/trec_eval.9.0.4/trec_eval'
# Essa função foi escrita usando o github copilot
def get_trec_eval_metrics(flags, qrels_path, results_path):
    """Runs trec_eval and returns the results as a dictionary.

    Args:
        flags (str): Flags to pass to trec_eval.
        qrels_path (str): Path to the qrels file.
        results_path (str): Path to the results file.

    Returns:
        Dict[str, float]: A dictionary mapping metric names to their values.
    """
    output = subprocess.check_output([
        TREC_EVAL_BIN_PATH,
        qrels_path,
        results_path,
        *shlex.split(flags)
    ]).decode('utf-8')
    return {
        line.split()[0]: (line.split()[2])
        for line in output.splitlines()
    }
# https://github.com/castorini/anserini/blob/master/docs/regressions-beir-v1.0.0-trec-covid-flat.md
# https://github.com/castorini/anserini/blob/master/src/main/resources/regression/beir-v1.0.0-trec-covid-flat.yaml



In [11]:
# -index /root/.cache/pyserini/indexes/lucene-index.beir-v1.0.0-trec-covid.flat.20221116.505594.57b812594b11d064a23123137ae7dade \
# -topics anserini/src/main/resources/topics-and-qrels/topics.beir-v1.0.0-trec-covid.test.tsv.gz \
!mkdir pv runs
!anserini/target/appassembler/bin/SearchCollection \
-index {PREBUILT_INDEX_DIR} \
-topics trec-covid-queries_trec_format.txt \
-topicreader TsvString \
-output runs/run.beir-v1.0.0-trec-covid-flat.bm25.topics.beir-v1.0.0-trec-covid.test.txt \
-bm25 -removeQuery -hits 1000

2023-04-11 03:21:38,982 INFO  [main] search.SearchCollection (SearchCollection.java:950) - ============ Initializing Searcher ============
2023-04-11 03:21:38,986 INFO  [main] search.SearchCollection (SearchCollection.java:951) - Index: /root/.cache/pyserini/indexes/lucene-index.beir-v1.0.0-trec-covid.flat.20221116.505594.57b812594b11d064a23123137ae7dade
2023-04-11 03:21:39,189 INFO  [main] search.SearchCollection (SearchCollection.java:955) - Fields: []
2023-04-11 03:21:39,190 INFO  [main] search.SearchCollection (SearchCollection.java:695) - Using DefaultEnglishAnalyzer
2023-04-11 03:21:39,191 INFO  [main] search.SearchCollection (SearchCollection.java:696) - Stemmer: porter
2023-04-11 03:21:39,192 INFO  [main] search.SearchCollection (SearchCollection.java:697) - Keep stopwords? false
2023-04-11 03:21:39,192 INFO  [main] search.SearchCollection (SearchCollection.java:698) - Stopwords file: null
2023-04-11 03:21:39,220 INFO  [main] search.SearchCollection (SearchCollection.java:1227)

In [12]:
#  -c: Average over the complete set of queries in the relevance judgements  
#      instead of the queries in the intersection of relevance judgements 
#      and results.  Missing queries will contribute a value of 0 to all 
#      evaluation measures (which may or may not be reasonable for a  
#      particular evaluation measure, but is reasonable for standard TREC
#      measures.) Default is off.
get_trec_eval_metrics('-c -m ndcg_cut.10', 'trec-covid-qrels_trec_format.txt', 'runs/run.beir-v1.0.0-trec-covid-flat.bm25.topics.beir-v1.0.0-trec-covid.test.txt')

{'ndcg_cut_10': '0.5947'}

The value for ndcg_cut_10 obtained match the expected one.

# Results using corpus expanded with generated queries

## Downloading files locally

In [13]:
!mkdir -pv ids_and_contents generated_queries
!gsutil -m rsync -r gs://marcospiau-doc2query-aula05/ids_and_contents ids_and_contents
!gsutil -m rsync -r gs://marcospiau-doc2query-aula05/generated_queries generated_queries

mkdir: created directory 'ids_and_contents'
mkdir: created directory 'generated_queries'
Building synchronization state...
Starting synchronization...
Copying gs://marcospiau-doc2query-aula05/ids_and_contents/one_line_per_document/content.txt...
Copying gs://marcospiau-doc2query-aula05/ids_and_contents/one_line_per_document/id.txt...
/ [2/2 files][185.1 MiB/185.1 MiB] 100% Done                                    
Operation completed over 2 objects/185.1 MiB.                                    
Building synchronization state...
Starting synchronization...
Copying gs://marcospiau-doc2query-aula05/generated_queries/one_line_per_document_beam_search/one_line_per_document_beam_search.log...
Copying gs://marcospiau-doc2query-aula05/generated_queries/one_line_per_document_topk_sampling/predictions-iter-14.txt-1005516...
Copying gs://marcospiau-doc2query-aula05/generated_queries/one_line_per_document_greedy_decode/one_line_per_document_greedy_decode.log...
Copying gs://marcospiau-doc2query-aul

In [14]:
!tree -lht ids_and_contents
!tree -lht generated_queries

ids_and_contents
└── [4.0K]  one_line_per_document
    ├── [184M]  content.txt
    └── [1.5M]  id.txt

1 directory, 2 files
generated_queries
├── [4.0K]  one_line_per_document_beam_search
│   ├── [6.8M]  predictions.txt-1005516
│   └── [412K]  one_line_per_document_beam_search.log
├── [4.0K]  one_line_per_document_greedy_decode
│   ├── [6.9M]  predictions.txt-1005516
│   └── [412K]  one_line_per_document_greedy_decode.log
└── [4.0K]  one_line_per_document_topk_sampling
    ├── [6.9M]  predictions-iter-01.txt-1005516
    ├── [7.0M]  predictions-iter-02.txt-1005516
    ├── [6.9M]  predictions-iter-03.txt-1005516
    ├── [7.0M]  predictions-iter-04.txt-1005516
    ├── [7.0M]  predictions-iter-05.txt-1005516
    ├── [7.0M]  predictions-iter-06.txt-1005516
    ├── [7.0M]  predictions-iter-07.txt-1005516
    ├── [7.0M]  predictions-iter-08.txt-1005516
    ├── [7.0M]  predictions-iter-09.txt-1005516
    ├── [7.0M]  predictions-iter-10.txt-1005516
    ├── [7.0M]  predictions-iter-11.txt-100551

There are unk tokens on generated queries:

In [15]:
!grep -ir ⁇ generated_queries | head

generated_queries/one_line_per_document_beam_search/one_line_per_document_beam_search.log:INFO:tensorflow:decoded 8: "Heme oxygenase-1 and carbon monoxide in pulmonary medicine. Heme oxygenase-1 (HO-1), an inducible stress protein, confers cytoprotection against oxidative stress in vitro and in vivo. In addition to its physiological role in heme degradation, HO-1 may influence a number of cellular processes, including growth, inflammation, and apoptosis. By virtue of anti-inflammatory effects, HO-1 limits tissue damage in response to proinflammatory stimuli and prevents allograft rejection after transplantation. The transcriptional upregulation of HO-1 responds to many agents, such as hypoxia, bacterial lipopolysaccharide, and reactive oxygen/nitrogen species. HO-1 and its constitutively expressed isozyme, heme oxygenase-2, catalyze the rate-limiting step in the conversion of heme to its metabolites, bilirubin IX ⁇ , ferrous iron, and carbon monoxide (CO). The mechanisms by which HO-1 

Replacing with sed:

In [16]:
!grep -ir ⁇ generated_queries | head | sed s,'⁇',' ',g

generated_queries/one_line_per_document_beam_search/one_line_per_document_beam_search.log:INFO:tensorflow:decoded 8: "Heme oxygenase-1 and carbon monoxide in pulmonary medicine. Heme oxygenase-1 (HO-1), an inducible stress protein, confers cytoprotection against oxidative stress in vitro and in vivo. In addition to its physiological role in heme degradation, HO-1 may influence a number of cellular processes, including growth, inflammation, and apoptosis. By virtue of anti-inflammatory effects, HO-1 limits tissue damage in response to proinflammatory stimuli and prevents allograft rejection after transplantation. The transcriptional upregulation of HO-1 responds to many agents, such as hypoxia, bacterial lipopolysaccharide, and reactive oxygen/nitrogen species. HO-1 and its constitutively expressed isozyme, heme oxygenase-2, catalyze the rate-limiting step in the conversion of heme to its metabolites, bilirubin IX   , ferrous iron, and carbon monoxide (CO). The mechanisms by which HO-1 

## Creating dataframe with original corpus and predictions

Data is small enough to be loaded in memory at once, so we will keep it simple and load all data in a single dataframe:

In [4]:
def load_ids(path):
    lines = []
    for line in open(path, 'r'):
        lines.append(line.strip())
    return lines

load_ids('ids_and_contents/one_line_per_document/id.txt')[:10]

['ug7v899j',
 '02tnwd4m',
 'ejv2xln0',
 '2b73a28n',
 '9785vg6d',
 'zjufx4fo',
 '5yhe786e',
 '8zchiykl',
 '8qnrcgnk',
 'jg13scgo']

In [5]:
def read_pred_files(*paths):
    def read_one(path):
        lines = []
        for line in open(path, 'r'):
            line = line.strip()
            line = line.replace('⁇', ' ')
            lines.append(line)
        return lines

    preds = list(map(read_one, paths))

    # assert all files have the same number of lines
    assert len(set(map(len, preds))) == 1
    # merge lines
    preds = list(zip(*preds))
    return preds

read_pred_files(*epath.Path('generated_queries/one_line_per_document_topk_sampling').glob('predictions-*'))[:10]

[('how many patients died from m pneumoniae',
  'how many children died from pneumonia',
  'characteristics of mycoplasma pneumoniae',
  'characteristics of M pneumoniae',
  'case series of mycoplasma pneumoniae',
  'how many patients died from pneumonia',
  'characteristics of mycoplasma pneumoniae in children',
  'series of prospective clinical trials on m pneumoniae',
  'characteristics of mycoplasma pneumoniae',
  'frequency of pneumonia in pre-school children',
  'what is the incidence of pneumonia in children',
  'when did the pneumonia come',
  'how many patients died from pneumonia',
  'how many patients died of pneumonia',
  'how many patients died from pneumonia',
  'how many patients died from m pneumonia',
  'how many patients died from pneumonia',
  'how many patients died from m pneumoniae',
  'how many children die from pneumonia',
  'how many patients died from pneumonia'),
 ('inflammatory pathways in lung disease',
  'what is NO• in the lung',
  'what is NO• in the lun

In [6]:
def load_ids_and_preds(ids_path, *preds_paths):
    ids = load_ids(ids_path)
    preds = read_pred_files(*preds_paths)
    assert len(ids) == len(preds)
    # ids can be duplicated, so we will groupby id and concat all preds for it
    # ids are sorted by construction
    out = {}
    for key, group in itertools.groupby(iterable=zip(ids, preds),
                                        key=lambda x: x[0]):
        out[key] = list(more_itertools.flatten(toolz.pluck(1, group)))
    return out

list(itertools.islice(load_ids_and_preds('ids_and_contents/one_line_per_document/id.txt', *epath.Path('generated_queries/one_line_per_document_topk_sampling').glob('predictions-*')).items(), 10))

[('ug7v899j',
  ['how many patients died from m pneumoniae',
   'how many children died from pneumonia',
   'characteristics of mycoplasma pneumoniae',
   'characteristics of M pneumoniae',
   'case series of mycoplasma pneumoniae',
   'how many patients died from pneumonia',
   'characteristics of mycoplasma pneumoniae in children',
   'series of prospective clinical trials on m pneumoniae',
   'characteristics of mycoplasma pneumoniae',
   'frequency of pneumonia in pre-school children',
   'what is the incidence of pneumonia in children',
   'when did the pneumonia come',
   'how many patients died from pneumonia',
   'how many patients died of pneumonia',
   'how many patients died from pneumonia',
   'how many patients died from m pneumonia',
   'how many patients died from pneumonia',
   'how many patients died from m pneumoniae',
   'how many children die from pneumonia',
   'how many patients died from pneumonia']),
 ('02tnwd4m',
  ['inflammatory pathways in lung disease',
   '

## Loading data

In [7]:
# single dataframe 
df_corpus_preds = pl.from_arrow(datasets.load_dataset(
    'BeIR/trec-covid','corpus')['corpus'].data.table)

# concat title and text - this is the baseline text for retrieval
# PS.: during query generation, we used the format
# pl.format('{}. {}', 'title', 'text').alias('title_and_text') to feed T5
# Here, we will simple concatenate the fields using a space

df_corpus_preds = df_corpus_preds.with_columns(
    pl.concat_str('title', 'text', separator=' ').alias('title_and_text'))
df_corpus_preds.head()

  0%|          | 0/1 [00:00<?, ?it/s]

_id,title,text,title_and_text
str,str,str,str
"""ug7v899j""","""Clinical features of culture-proven Mycoplasma pneumoniae infections at King Abdulaziz University Hospital, Jeddah, Saudi Arabia""","""OBJECTIVE: This retrospective chart review describes the epidemiology and clinical features of 40 patients with culture-proven Mycoplasma pneumoniae infections at King Abdulaziz University Hospital, Jeddah, Saudi Arabia. METHODS: Patients with positive M. pneumoniae cultures from respiratory specimens from January 1997 through December 1998 were identified through the Microbiology records. Charts of patients were reviewed. RESULTS: 40 patients were identified, 33 (82.5%) of whom required admission. Most infections (92.5%) were community-acquired. The infection affected all age groups but was most common in infants (32.5%) and pre-school children (22.5%). It occurred year-round but was most common in the fall (35%) and spring (30%). More than three-quarters of patients (77.5%) had comorbidities. Twenty-four isolates (60%) were associated with pneumonia, 14 (35%) with upper respiratory tract infections, and 2 (5%) with bronchiolitis. Cough (82.5%), fever (75%), and malaise (58.8%) were …","""Clinical features of culture-proven Mycoplasma pneumoniae infections at King Abdulaziz University Hospital, Jeddah, Saudi Arabia OBJECTIVE: This retrospective chart review describes the epidemiology and clinical features of 40 patients with culture-proven Mycoplasma pneumoniae infections at King Abdulaziz University Hospital, Jeddah, Saudi Arabia. METHODS: Patients with positive M. pneumoniae cultures from respiratory specimens from January 1997 through December 1998 were identified through the Microbiology records. Charts of patients were reviewed. RESULTS: 40 patients were identified, 33 (82.5%) of whom required admission. Most infections (92.5%) were community-acquired. The infection affected all age groups but was most common in infants (32.5%) and pre-school children (22.5%). It occurred year-round but was most common in the fall (35%) and spring (30%). More than three-quarters of patients (77.5%) had comorbidities. Twenty-four isolates (60%) were associated with pneumonia, 14 (3…"
"""02tnwd4m""","""Nitric oxide: a pro-inflammatory mediator in lung disease?""","""Inflammatory diseases of the respiratory tract are commonly associated with elevated production of nitric oxide (NO•) and increased indices of NO• -dependent oxidative stress. Although NO• is known to have anti-microbial, anti-inflammatory and anti-oxidant properties, various lines of evidence support the contribution of NO• to lung injury in several disease models. On the basis of biochemical evidence, it is often presumed that such NO• -dependent oxidations are due to the formation of the oxidant peroxynitrite, although alternative mechanisms involving the phagocyte-derived heme proteins myeloperoxidase and eosinophil peroxidase might be operative during conditions of inflammation. Because of the overwhelming literature on NO• generation and activities in the respiratory tract, it would be beyond the scope of this commentary to review this area comprehensively. Instead, it focuses on recent evidence and concepts of the presumed contribution of NO• to inflammatory diseases of the lun…","""Nitric oxide: a pro-inflammatory mediator in lung disease? Inflammatory diseases of the respiratory tract are commonly associated with elevated production of nitric oxide (NO•) and increased indices of NO• -dependent oxidative stress. Although NO• is known to have anti-microbial, anti-inflammatory and anti-oxidant properties, various lines of evidence support the contribution of NO• to lung injury in several disease models. On the basis of biochemical evidence, it is often presumed that such NO• -dependent oxidations are due to the formation of the oxidant peroxynitrite, although alternative mechanisms involving the phagocyte-derived heme proteins myeloperoxidase and eosinophil peroxidase might be

Adding generated queries:

In [8]:
preds_paths_map = {
    x.name: ('ids_and_contents/one_line_per_document/id.txt',
             sorted(x.glob('predictions*')))
    for x in epath.Path('generated_queries').iterdir()
}
preds_paths_map

{'one_line_per_document_beam_search': ('ids_and_contents/one_line_per_document/id.txt',
  [PosixGPath('generated_queries/one_line_per_document_beam_search/predictions.txt-1005516')]),
 'one_line_per_document_topk_sampling': ('ids_and_contents/one_line_per_document/id.txt',
  [PosixGPath('generated_queries/one_line_per_document_topk_sampling/predictions-iter-01.txt-1005516'),
   PosixGPath('generated_queries/one_line_per_document_topk_sampling/predictions-iter-02.txt-1005516'),
   PosixGPath('generated_queries/one_line_per_document_topk_sampling/predictions-iter-03.txt-1005516'),
   PosixGPath('generated_queries/one_line_per_document_topk_sampling/predictions-iter-04.txt-1005516'),
   PosixGPath('generated_queries/one_line_per_document_topk_sampling/predictions-iter-05.txt-1005516'),
   PosixGPath('generated_queries/one_line_per_document_topk_sampling/predictions-iter-06.txt-1005516'),
   PosixGPath('generated_queries/one_line_per_document_topk_sampling/predictions-iter-07.txt-1005516')

In [9]:
df_corpus_preds = df_corpus_preds.with_columns(
    pl.col('_id').map_dict(load_ids_and_preds(v[0], *v[1])).alias(f'preds_{k}')
    for k, v in tqdm(preds_paths_map.items())
)

100%|██████████| 3/3 [00:04<00:00,  1.53s/it]


In [56]:
!mkdir -pv backup_dir
df_corpus_preds.write_ipc('backup_dir/df_corpus_preds.arrow')
!ls -lht backup_dir

mkdir: created directory 'backup_dir'
total 555M
-rw-r--r-- 1 root root 555M Apr 11 06:18 df_corpus_preds.arrow


## Creating jsonl files for indexing

In [11]:
def prepare_data_for_indexing(df, col_with_preds=None, max_preds=None, col_with_text='title_and_text'):
    to_concat = []
    if col_with_text is not None:
        to_concat.append(col_with_text)
    if col_with_preds is not None:
        preds_expr = pl.col(col_with_preds)
        if max_preds is not None:
            preds_expr = preds_expr.arr.slice(0, max_preds)
        to_concat.append(preds_expr.arr.join(' '))

    df = df.select(
        pl.col('_id').alias('id'),
        pl.concat_str(to_concat, separator= ' ').alias('contents'))
    return df

prepare_data_for_indexing(df_corpus_preds, 'preds_one_line_per_document_topk_sampling').head(1)

id,contents
str,str
"""ug7v899j""","""Clinical features of culture-proven Mycoplasma pneumoniae infections at King Abdulaziz University Hospital, Jeddah, Saudi Arabia OBJECTIVE: This retrospective chart review describes the epidemiology and clinical features of 40 patients with culture-proven Mycoplasma pneumoniae infections at King Abdulaziz University Hospital, Jeddah, Saudi Arabia. METHODS: Patients with positive M. pneumoniae cultures from respiratory specimens from January 1997 through December 1998 were identified through the Microbiology records. Charts of patients were reviewed. RESULTS: 40 patients were identified, 33 (82.5%) of whom required admission. Most infections (92.5%) were community-acquired. The infection affected all age groups but was most common in infants (32.5%) and pre-school children (22.5%). It occurred year-round but was most common in the fall (35%) and spring (30%). More than three-quarters of patients (77.5%) had comorbidities. Twenty-four isolates (60%) were associated with pneumonia, 14 (3…"


In [41]:
# index with expanded queries_only
prepare_data_for_indexing_expansion_only = functools.partial(
    prepare_data_for_indexing,
    col_with_text=None
)

prepare_data_for_indexing_no_expansion = functools.partial(
    prepare_data_for_indexing,
    col_with_preds=None,
    max_preds=None
)

In [43]:
index_build_fns = {
    'title_and_text_no_expansion': prepare_data_for_indexing_no_expansion}

for pred_col in ['preds_one_line_per_document_greedy_decode', 'preds_one_line_per_document_beam_search', 'preds_one_line_per_document_topk_sampling']:
    if pred_col == 'preds_one_line_per_document_topk_sampling':
        max_preds_range = range(1, 21)
    else:
        max_preds_range = range(1, 2)

    for max_preds in max_preds_range:
        # text and expansion
        index_build_fns[f'{pred_col}_{max_preds}_pred'] = functools.partial(
            prepare_data_for_indexing,
            col_with_preds=pred_col,
            max_preds=max_preds,
            col_with_text='title_and_text')
        # only expansion
        index_build_fns[f'only_preds_{pred_col}_{max_preds}_pred'] = functools.partial(
            prepare_data_for_indexing_expansion_only,
            col_with_preds=pred_col,
            max_preds=max_preds
        )

    del pred_col

len(index_build_fns)

45

In [46]:
index_build_fns['only_preds_preds_one_line_per_document_topk_sampling_18_pred'](df_corpus_preds).head()

shape: (5, 2)
┌──────────┬───────────────────────────────────────────────────────────────────────────────────────┐
│ id       ┆ contents                                                                              │
│ ---      ┆ ---                                                                                   │
│ str      ┆ str                                                                                   │
╞══════════╪═══════════════════════════════════════════════════════════════════════════════════════╡
│ ug7v899j ┆ when did the pneumonia come how many children die from pneumonia characteristics of M │
│          ┆ pneumoniae characteristics of mycoplasma pneumoniae in children how many patients     │
│          ┆ died from pneumonia characteristics of mycoplasma pneumoniae series of prospective    │
│          ┆ clinical trials on m pneumoniae how many children died from pneumonia how many        │
│          ┆ patients died from m pneumoniae characteristics of mycoplasma pn

In [49]:
!mkdir -pv jsonl_for_indexing

jsonl_for_indexing = {}
for key, fn in tqdm(index_build_fns.items(),
                    desc='Writing jsonl files for indexing'):
    jsonl_path = Path('jsonl_for_indexing').joinpath(key, 'corpus.jsonl')
    jsonl_path.parent.mkdir(parents=True, exist_ok=True)

    fn(df_corpus_preds).write_ndjson(jsonl_path)
    jsonl_for_indexing[key] = str(jsonl_path)
!tree -lht jsonl_for_indexing

mkdir: created directory 'jsonl_for_indexing'


Writing jsonl files for indexing: 100%|██████████| 45/45 [00:51<00:00,  1.15s/it]


jsonl_for_indexing
├── [4.0K]  title_and_text_no_expansion
│   └── [188M]  corpus.jsonl
├── [4.0K]  preds_one_line_per_document_greedy_decode_1_pred
│   └── [195M]  corpus.jsonl
├── [4.0K]  only_preds_preds_one_line_per_document_greedy_decode_1_pred
│   └── [ 12M]  corpus.jsonl
├── [4.0K]  preds_one_line_per_document_beam_search_1_pred
│   └── [195M]  corpus.jsonl
├── [4.0K]  only_preds_preds_one_line_per_document_beam_search_1_pred
│   └── [ 12M]  corpus.jsonl
├── [4.0K]  preds_one_line_per_document_topk_sampling_1_pred
│   └── [195M]  corpus.jsonl
├── [4.0K]  only_preds_preds_one_line_per_document_topk_sampling_1_pred
│   └── [ 12M]  corpus.jsonl
├── [4.0K]  preds_one_line_per_document_topk_sampling_2_pred
│   └── [202M]  corpus.jsonl
├── [4.0K]  only_preds_preds_one_line_per_document_topk_sampling_2_pred
│   └── [ 19M]  corpus.jsonl
├── [4.0K]  preds_one_line_per_document_topk_sampling_3_pred
│   └── [209M]  corpus.jsonl
├── [4.0K]  only_preds_preds_one_line_per_document_topk_sampli

## Creating indexes with anserini

In [50]:
import shlex
import subprocess

ANSERINI_INDEX_COLLECTION_BIN_PATH = 'anserini/target/appassembler/bin/IndexCollection'
DEFAULT_INDEXING_FLAGS = (
    '-collection JsonCollection -generator DefaultLuceneDocumentGenerator '
    '-threads 1 -storePositions -storeDocvectors -storeRaw -optimize')
def create_anserini_index(input_dir, output_dir, flags=DEFAULT_INDEXING_FLAGS):
    """Index jsonl files using anserini.

    Args:
        input_dir: input dir with json corpus
        output_dir: output_dir for index
        flags (str): Flags to pass to IndexCollection binary
    
    Returns:
        command stdout
    """
    output = subprocess.check_output([
        ANSERINI_INDEX_COLLECTION_BIN_PATH,
        '-input', input_dir,
        '-index', output_dir,
        *shlex.split(flags)
    ]).decode('utf-8')
    return output

In [51]:
# args will be input_dir and output_dir
index_args = list(map(lambda x: (x, Path('indexes', x.name)),
                      Path('jsonl_for_indexing').iterdir()))
index_args

[(PosixPath('jsonl_for_indexing/preds_one_line_per_document_topk_sampling_5_pred'),
  PosixPath('indexes/preds_one_line_per_document_topk_sampling_5_pred')),
 (PosixPath('jsonl_for_indexing/only_preds_preds_one_line_per_document_topk_sampling_12_pred'),
  PosixPath('indexes/only_preds_preds_one_line_per_document_topk_sampling_12_pred')),
 (PosixPath('jsonl_for_indexing/preds_one_line_per_document_topk_sampling_19_pred'),
  PosixPath('indexes/preds_one_line_per_document_topk_sampling_19_pred')),
 (PosixPath('jsonl_for_indexing/only_preds_preds_one_line_per_document_topk_sampling_1_pred'),
  PosixPath('indexes/only_preds_preds_one_line_per_document_topk_sampling_1_pred')),
 (PosixPath('jsonl_for_indexing/only_preds_preds_one_line_per_document_topk_sampling_15_pred'),
  PosixPath('indexes/only_preds_preds_one_line_per_document_topk_sampling_15_pred')),
 (PosixPath('jsonl_for_indexing/preds_one_line_per_document_topk_sampling_14_pred'),
  PosixPath('indexes/preds_one_line_per_document_topk

In [52]:
%%time
!mkdir -pv indexes
N_WORKERS = 16
with mp.Pool(N_WORKERS) as pool:
    print(f'Indexing using a process pool of {N_WORKERS} workers')
    stdouts_indexing = pool.starmap(create_anserini_index, index_args)

mkdir: created directory 'indexes'
Indexing using a process pool of 16 workers
CPU times: user 1.02 s, sys: 1.38 s, total: 2.41 s
Wall time: 2min 34s


In [53]:
df_index_stdouts = pl.DataFrame([
    pl.Series(list(map(lambda x: x[0].name, index_args))).alias('index'),
    pl.Series(stdouts_indexing).alias('stdout')]
)
df_index_stdouts.head()

index,stdout
str,str
"""preds_one_line_per_document_topk_sampling_5_pred""","""2023-04-11 06:13:56,006 INFO [main] index.IndexCollection (IndexCollection.java:380) - Setting log level to INFO 2023-04-11 06:13:56,010 INFO [main] index.IndexCollection (IndexCollection.java:383) - Starting indexer... 2023-04-11 06:13:56,010 INFO [main] index.IndexCollection (IndexCollection.java:384) - ============ Loading Parameters ============ 2023-04-11 06:13:56,011 INFO [main] index.IndexCollection (IndexCollection.java:385) - DocumentCollection path: jsonl_for_indexing/preds_one_line_per_document_topk_sampling_5_pred 2023-04-11 06:13:56,012 INFO [main] index.IndexCollection (IndexCollection.java:386) - CollectionClass: JsonCollection 2023-04-11 06:13:56,013 INFO [main] index.IndexCollection (IndexCollection.java:387) - Generator: DefaultLuceneDocumentGenerator 2023-04-11 06:13:56,013 INFO [main] index.IndexCollection (IndexCollection.java:388) - Threads: 1 2023-04-11 06:13:56,014 INFO [main] index.IndexCollection (IndexCollection.java:389) - Language: en 2023-04-11 06…"
"""only_preds_preds_one_line_per_document_topk_sampling_12_pred""","""2023-04-11 06:13:55,940 INFO [main] index.IndexCollection (IndexCollection.java:380) - Setting log level to INFO 2023-04-11 06:13:55,944 INFO [main] index.IndexCollection (IndexCollection.java:383) - Starting indexer... 2023-04-11 06:13:55,944 INFO [main] index.IndexCollection (IndexCollection.java:384) - ============ Loading Parameters ============ 2023-04-11 06:13:55,945 INFO [main] index.IndexCollection (IndexCollection.java:385) - DocumentCollection path: jsonl_for_indexing/only_preds_preds_one_line_per_document_topk_sampling_12_pred 2023-04-11 06:13:55,946 INFO [main] index.IndexCollection (IndexCollection.java:386) - CollectionClass: JsonCollection 2023-04-11 06:13:55,948 INFO [main] index.IndexCollection (IndexCollection.java:387) - Generator: DefaultLuceneDocumentGenerator 2023-04-11 06:13:55,949 INFO [main] index.IndexCollection (IndexCollection.java:388) - Threads: 1 2023-04-11 06:13:55,950 INFO [main] index.IndexCollection (IndexCollection.java:389) - Language: en 2…"
"""preds_one_line_per_document_topk_sampling_19_pred""","""2023-04-11 06:13:55,991 INFO [main] index.IndexCollection (IndexCollection.java:380) - Setting log level to INFO 2023-04-11 06:13:55,995 INFO [main] index.IndexCollection (IndexCollection.java:383) - Starting indexer... 2023-04-11 06:13:55,996 INFO [main] index.IndexCollection (IndexCollection.java:384) - ============ Loading Parameters ============ 2023-04-11 06:13:55,997 INFO [main] index.IndexCollection (IndexCollection.java:385) - DocumentCollection path: jsonl_for_indexing/preds_one_line_per_document_topk_sampling_19_pred 2023-04-11 06:13:55,997 INFO [main] index.IndexCollection (IndexCollection.java:386) - CollectionClass: JsonCollection 2023-04-11 06:13:55,999 INFO [main] index.IndexCollection (IndexCollection.java:387) - Generator: DefaultLuceneDocumentGenerator 2023-04-11 06:13:56,000 INFO [main] index.IndexCollection (IndexCollection.java:388) - Threads: 1 2023-04-11 06:13:56,000 INFO [main] index.IndexCollection (IndexCollection.java:389) - Language: en 2023-04-11 0…"
"""only_preds_preds_one_line_per_document_topk_sampling_1_pred""","""2023-04-11 06:13:55,918 INFO [main] index.IndexCollection (IndexCollection.java:380) - Setting log level to INFO 2023-04-11 06:13:55,923 INFO [main] index.IndexCollection (IndexCollection.java:383) - Starting indexer... 2023-04-11 06:13:55,923 INFO [main] index.IndexCollection (IndexCollection.java:384) - ============ Loading Parameters ============ 2023-04-11 06:13:55,924 INFO [main] index.IndexCollection (IndexCollection.java:385) - DocumentCollection path: jsonl_for_indexing/only_preds_preds_one_line_per_document_topk_sampling_1_pred 2023-04-11 06:13:55,925 INFO [main] index.IndexCollection (IndexCollection.java:386) - CollectionClass: JsonCollection 2023-04-11 06:13:55,927 INFO [main] index.IndexCollection (IndexCollection.java

In [57]:
df_index_stdouts.write_ndjson('backup_dir/index_building_stdouts.jsonl')
!head backup_dir/index_building_stdouts.jsonl

{"index":"preds_one_line_per_document_topk_sampling_5_pred","stdout":"2023-04-11 06:13:56,006 INFO  [main] index.IndexCollection (IndexCollection.java:380) - Setting log level to INFO\n2023-04-11 06:13:56,010 INFO  [main] index.IndexCollection (IndexCollection.java:383) - Starting indexer...\n2023-04-11 06:13:56,010 INFO  [main] index.IndexCollection (IndexCollection.java:384) - ============ Loading Parameters ============\n2023-04-11 06:13:56,011 INFO  [main] index.IndexCollection (IndexCollection.java:385) - DocumentCollection path: jsonl_for_indexing/preds_one_line_per_document_topk_sampling_5_pred\n2023-04-11 06:13:56,012 INFO  [main] index.IndexCollection (IndexCollection.java:386) - CollectionClass: JsonCollection\n2023-04-11 06:13:56,013 INFO  [main] index.IndexCollection (IndexCollection.java:387) - Generator: DefaultLuceneDocumentGenerator\n2023-04-11 06:13:56,013 INFO  [main] index.IndexCollection (IndexCollection.java:388) - Threads: 1\n2023-04-11 06:13:56,014 INFO  [main] i

## Generating runs (retrieval)

In [58]:
ANSERINI_SEARCH_COLLECTION_BIN_PATH = 'anserini/target/appassembler/bin/SearchCollection'
DEFAULT_SEARCH_FLAGS = '-topicreader TsvString -bm25 -removeQuery -hits 1000'
def create_run(index_dir, topics_path, results_path,
               flags=DEFAULT_SEARCH_FLAGS):
    """Generate results runs using anserini.

    Args:
        index_dir (str): directory with index
        topics_path (str): path to topics (queries)
        output_file (str): output file with runs (results)
        flags (str): Flags to pass to SearchCollection binary.
    
    Returns:
        command stdout
    """
    output = subprocess.check_output([
        ANSERINI_SEARCH_COLLECTION_BIN_PATH,
        '-index', index_dir,
        '-topics', topics_path,
        '-output', results_path,
        *shlex.split(flags)
    ]).decode('utf-8')
    return output

In [59]:
%%time
!mkdir -pv runs/

create_runs_stdouts = []
for index_dir in tqdm(list(Path('indexes').iterdir())):
    create_runs_stdouts.append({
        'index_dir': index_dir.name,
        'search_results_stdout': create_run(
            index_dir=index_dir,
            topics_path='trec-covid-queries_trec_format.txt',
            results_path=Path('runs') / f'run.{index_dir.name}.txt'
        )
    })

mkdir: created directory 'runs/'


100%|██████████| 45/45 [01:55<00:00,  2.57s/it]

CPU times: user 883 ms, sys: 4.75 s, total: 5.63 s
Wall time: 1min 55s


In [60]:
df_create_runs_stdout = pl.DataFrame(create_runs_stdouts)
df_create_runs_stdout = df_create_runs_stdout.with_columns(
    pl.col('search_results_stdout').str.extract('(.*?)~(.*?) q/s', 2)
    .apply(float).alias('retrieval_queries_per_second'))
df_create_runs_stdout

index_dir,search_results_stdout,retrieval_queries_per_second
str,str,f64
"""preds_one_line_per_document_topk_sampling_5_pred""","""2023-04-11 06:18:37,835 INFO [main] search.SearchCollection (SearchCollection.java:950) - ============ Initializing Searcher ============ 2023-04-11 06:18:37,839 INFO [main] search.SearchCollection (SearchCollection.java:951) - Index: indexes/preds_one_line_per_document_topk_sampling_5_pred 2023-04-11 06:18:38,034 INFO [main] search.SearchCollection (SearchCollection.java:955) - Fields: [] 2023-04-11 06:18:38,035 INFO [main] search.SearchCollection (SearchCollection.java:695) - Using DefaultEnglishAnalyzer 2023-04-11 06:18:38,036 INFO [main] search.SearchCollection (SearchCollection.java:696) - Stemmer: porter 2023-04-11 06:18:38,036 INFO [main] search.SearchCollection (SearchCollection.java:697) - Keep stopwords? false 2023-04-11 06:18:38,037 INFO [main] search.SearchCollection (SearchCollection.java:698) - Stopwords file: null 2023-04-11 06:18:38,063 INFO [main] search.SearchCollection (SearchCollection.java:1227) - runtag: Anserini 2023-04-11 06:18:38,066 INFO [main] searc…",32.53
"""only_preds_preds_one_line_per_document_topk_sampling_12_pred""","""2023-04-11 06:18:40,576 INFO [main] search.SearchCollection (SearchCollection.java:950) - ============ Initializing Searcher ============ 2023-04-11 06:18:40,580 INFO [main] search.SearchCollection (SearchCollection.java:951) - Index: indexes/only_preds_preds_one_line_per_document_topk_sampling_12_pred 2023-04-11 06:18:40,769 INFO [main] search.SearchCollection (SearchCollection.java:955) - Fields: [] 2023-04-11 06:18:40,770 INFO [main] search.SearchCollection (SearchCollection.java:695) - Using DefaultEnglishAnalyzer 2023-04-11 06:18:40,771 INFO [main] search.SearchCollection (SearchCollection.java:696) - Stemmer: porter 2023-04-11 06:18:40,771 INFO [main] search.SearchCollection (SearchCollection.java:697) - Keep stopwords? false 2023-04-11 06:18:40,772 INFO [main] search.SearchCollection (SearchCollection.java:698) - Stopwords file: null 2023-04-11 06:18:40,799 INFO [main] search.SearchCollection (SearchCollection.java:1227) - runtag: Anserini 2023-04-11 06:18:40,802 INFO …",45.5
"""preds_one_line_per_document_topk_sampling_19_pred""","""2023-04-11 06:18:42,853 INFO [main] search.SearchCollection (SearchCollection.java:950) - ============ Initializing Searcher ============ 2023-04-11 06:18:42,857 INFO [main] search.SearchCollection (SearchCollection.java:951) - Index: indexes/preds_one_line_per_document_topk_sampling_19_pred 2023-04-11 06:18:43,047 INFO [main] search.SearchCollection (SearchCollection.java:955) - Fields: [] 2023-04-11 06:18:43,048 INFO [main] search.SearchCollection (SearchCollection.java:695) - Using DefaultEnglishAnalyzer 2023-04-11 06:18:43,049 INFO [main] search.SearchCollection (SearchCollection.java:696) - Stemmer: porter 2023-04-11 06:18:43,049 INFO [main] search.SearchCollection (SearchCollection.java:697) - Keep stopwords? false 2023-04-11 06:18:43,050 INFO [main] search.SearchCollection (SearchCollection.java:698) - Stopwords file: null 2023-04-11 06:18:43,074 INFO [main] search.SearchCollection (SearchCollection.java:1227) - runtag: Anserini 2023-04-11 06:18:43,077 INFO [main] sear…",28.52
"""only_preds_preds_one_line_per_document_topk_sampling_1_pred""","""2023-04-11 06:18:45,855 INFO [main] search.SearchCollection (SearchCollection.java:950) - ============ Initializing Searcher ============ 2023-04-11 06:18:45,858 INFO [main] search.SearchCollection (SearchCollection.java:951) - Index: indexes/only_preds_preds_one_line_per_document_topk_sampling_1_pred 2023-04-11 06:18:46,045 INFO [main] search.SearchCollection (SearchCollection.java:955) - Fields: [] 2023-04-11 06:18:46,046 INFO [main] search.SearchCollection (SearchCollection.java:695) - Using DefaultEnglishAnalyzer 2023-04-11 06:18:46,047 INFO [main] search.SearchCollection (SearchCollection.java:696) - Stemmer: porter 2023-04-11 06:18:46,047 INFO [main] search.Sea

In [61]:
df_create_runs_stdout.write_ndjson('backup_dir/df_create_runs_stdout.jsonl')
!head backup_dir/df_create_runs_stdout.jsonl
!ls -lht backup_dir

{"index_dir":"preds_one_line_per_document_topk_sampling_5_pred","search_results_stdout":"2023-04-11 06:18:37,835 INFO  [main] search.SearchCollection (SearchCollection.java:950) - ============ Initializing Searcher ============\n2023-04-11 06:18:37,839 INFO  [main] search.SearchCollection (SearchCollection.java:951) - Index: indexes/preds_one_line_per_document_topk_sampling_5_pred\n2023-04-11 06:18:38,034 INFO  [main] search.SearchCollection (SearchCollection.java:955) - Fields: []\n2023-04-11 06:18:38,035 INFO  [main] search.SearchCollection (SearchCollection.java:695) - Using DefaultEnglishAnalyzer\n2023-04-11 06:18:38,036 INFO  [main] search.SearchCollection (SearchCollection.java:696) - Stemmer: porter\n2023-04-11 06:18:38,036 INFO  [main] search.SearchCollection (SearchCollection.java:697) - Keep stopwords? false\n2023-04-11 06:18:38,037 INFO  [main] search.SearchCollection (SearchCollection.java:698) - Stopwords file: null\n2023-04-11 06:18:38,063 INFO  [main] search.SearchCollec

## Calculating metrics for runs

In [62]:
eval_metrics_fn = functools.partial(
    get_trec_eval_metrics,
    '-c -m ndcg_cut.10',
    'trec-covid-qrels_trec_format.txt'    
)

In [63]:
%%time
df_results = pl.Series(map(str, Path('runs').iterdir())).to_frame('run_path')
df_results = df_results.with_columns(pl.col('run_path').apply(eval_metrics_fn).alias('metrics'))
df_results = (df_results
              .unnest('metrics')
              .with_columns(pl.col('ndcg_cut_10').apply(float)))
df_results = df_results.with_columns(pl.col('run_path').str.extract('/run.(.*?).txt').alias('run_name'))
df_results = df_results.join(
    df_create_runs_stdout.select(pl.col('index_dir').alias('run_name'),
                                 'retrieval_queries_per_second'),
                             on='run_name', how='left')
df_results

CPU times: user 163 ms, sys: 2.88 s, total: 3.04 s
Wall time: 8.86 s


run_path,ndcg_cut_10,run_name,retrieval_queries_per_second
str,f64,str,f64
"""runs/run.only_preds_preds_one_line_per_document_topk_sampling_5_pred.txt""",0.4565,"""only_preds_preds_one_line_per_document_topk_sampling_5_pred""",43.98
"""runs/run.preds_one_line_per_document_topk_sampling_20_pred.txt""",0.6748,"""preds_one_line_per_document_topk_sampling_20_pred""",32.18
"""runs/run.title_and_text_no_expansion.txt""",0.5947,"""title_and_text_no_expansion""",37.97
"""runs/run.only_preds_preds_one_line_per_document_topk_sampling_3_pred.txt""",0.4398,"""only_preds_preds_one_line_per_document_topk_sampling_3_pred""",36.21
"""runs/run.preds_one_line_per_document_topk_sampling_3_pred.txt""",0.6489,"""preds_one_line_per_document_topk_sampling_3_pred""",31.89
"""runs/run.preds_one_line_per_document_topk_sampling_5_pred.txt""",0.6599,"""preds_one_line_per_document_topk_sampling_5_pred""",32.53
"""runs/run.only_preds_preds_one_line_per_document_beam_search_1_pred.txt""",0.3384,"""only_preds_preds_one_line_per_document_beam_search_1_pred""",42.02
"""runs/run.preds_one_line_per_document_topk_sampling_2_pred.txt""",0.6427,"""preds_one_line_per_document_topk_sampling_2_pred""",34.39
"""runs/run.only_preds_preds_one_line_per_document_topk_sampling_7_pred.txt""",0.4865,"""only_preds_preds_one_line_per_document_topk_sampling_7_pred""",41.25


In [71]:
df_results.sort('ndcg_cut_10').to_pandas()

,run_path,ndcg_cut_10,run_name,retrieval_queries_per_second
0,runs/run.only_preds_preds_one_line_per_document_beam_search_1_pred.txt,0.3384,only_preds_preds_one_line_per_document_beam_search_1_pred,42.02
1,runs/run.only_preds_preds_one_line_per_document_greedy_decode_1_pred.txt,0.3523,only_preds_preds_one_line_per_document_greedy_decode_1_pred,37.76
2,runs/run.only_preds_preds_one_line_per_document_topk_sampling_1_pred.txt,0.3557,only_preds_preds_one_line_per_document_topk_sampling_1_pred,43.44
3,runs/run.only_preds_preds_one_line_per_document_topk_sampling_2_pred.txt,0.4191,only_preds_preds_one_line_per_document_topk_sampling_2_pred,38.20
4,runs/run.only_preds_preds_one_line_per_document_topk_sampling_3_pred.txt,0.4398,only_preds_preds_one_line_per_document_topk_sampling_3_pred,36.21
5,runs/run.only_preds_preds_one_line_per_document_topk_sampling_4_pred.txt,0.4507,only_preds_preds_one_line_per_document_topk_sampling_4_pred,52.41
6,runs/run.only_preds_preds_one_line_per_document_topk_sampling_5_pred.txt,0.4565,only_preds_preds_one_line_per_document_topk_sampling_5_pred,43.98
7,runs/run.only_preds_preds_one_line_per_document_topk_sampling_6_pred.txt,0.4800,only_preds_preds_one_line_per_document_topk_sampling_6_pred,42.96
8,runs/run.only_preds_preds_one_line_per_document_topk_sampling_7_pred.txt,0.4865,only_preds_preds_one_line_per_document_topk_sampling_7_pred,41.25
9,runs/run.only_preds_preds_one_line_per_document_topk_sampling_10_pred.txt,0.4976,only_preds_preds_one_line_per_document_topk_sampling_10_pred,41.60


In [73]:
df_results.write_csv()

'run_path,ndcg_cut_10,run_name,retrieval_queries_per_second\nruns/run.only_preds_preds_one_line_per_document_topk_sampling_5_pred.txt,0.4565,only_preds_preds_one_line_per_document_topk_sampling_5_pred,43.98\nruns/run.preds_one_line_per_document_topk_sampling_20_pred.txt,0.6748,preds_one_line_per_document_topk_sampling_20_pred,32.18\nruns/run.title_and_text_no_expansion.txt,0.5947,title_and_text_no_expansion,37.97\nruns/run.only_preds_preds_one_line_per_document_topk_sampling_3_pred.txt,0.4398,only_preds_preds_one_line_per_document_topk_sampling_3_pred,36.21\nruns/run.preds_one_line_per_document_topk_sampling_3_pred.txt,0.6489,preds_one_line_per_document_topk_sampling_3_pred,31.89\nruns/run.preds_one_line_per_document_topk_sampling_5_pred.txt,0.6599,preds_one_line_per_document_topk_sampling_5_pred,32.53\nruns/run.only_preds_preds_one_line_per_document_beam_search_1_pred.txt,0.3384,only_preds_preds_one_line_per_document_beam_search_1_pred,42.02\nruns/run.preds_one_line_per_document_topk_

In [88]:
df_results.write_ndjson('backup_dir/df_results.jsonl')
!ls -lht backup_dir

total 555M
-rw-r--r-- 1 root root 9.2K Apr 11 06:29 df_results.jsonl
-rw-r--r-- 1 root root  70K Apr 11 06:20 df_create_runs_stdout.jsonl
-rw-r--r-- 1 root root 203K Apr 11 06:18 index_building_stdouts.jsonl
-rw-r--r-- 1 root root 555M Apr 11 06:18 df_corpus_preds.arrow


In [91]:
!cp -v trec-covid-queries_trec_format.txt trec-covid-qrels_trec_format.txt backup_dir
!ls -lht backup_dir

'trec-covid-queries_trec_format.txt' -> 'backup_dir/trec-covid-queries_trec_format.txt'
'trec-covid-qrels_trec_format.txt' -> 'backup_dir/trec-covid-qrels_trec_format.txt'
total 556M
-rw-r--r-- 1 root root 1023K Apr 11 06:30 trec-covid-qrels_trec_format.txt
-rw-r--r-- 1 root root  3.6K Apr 11 06:30 trec-covid-queries_trec_format.txt
-rw-r--r-- 1 root root  9.2K Apr 11 06:29 df_results.jsonl
-rw-r--r-- 1 root root   70K Apr 11 06:20 df_create_runs_stdout.jsonl
-rw-r--r-- 1 root root  203K Apr 11 06:18 index_building_stdouts.jsonl
-rw-r--r-- 1 root root  555M Apr 11 06:18 df_corpus_preds.arrow


## Backing up data to GCP for later

In [93]:
%%time
!gsutil -m rsync -r indexes gs://marcospiau-doc2query-aula05/indexes
!gsutil -m rsync -r runs gs://marcospiau-doc2query-aula05/runs
!gsutil -m rsync -r backup_dir gs://marcospiau-doc2query-aula05/backup_dir
!gsutil -m rsync -r jsonl_for_indexing gs://marcospiau-doc2query-aula05/jsonl_for_indexing

Building synchronization state...
Starting synchronization...
Copying file://indexes/only_preds_preds_one_line_per_document_beam_search_1_pred/_0.fdm [Content-Type=application/octet-stream]...
Copying file://indexes/only_preds_preds_one_line_per_document_greedy_decode_1_pred/_0.fdx [Content-Type=application/octet-stream]...
Copying file://indexes/only_preds_preds_one_line_per_document_beam_search_1_pred/_0.tvm [Content-Type=application/octet-stream]...
Copying file://indexes/only_preds_preds_one_line_per_document_greedy_decode_1_pred/_0.fdt [Content-Type=application/octet-stream]...
Copying file://indexes/only_preds_preds_one_line_per_document_beam_search_1_pred/_0_Lucene90_0.dvm [Content-Type=application/octet-stream]...
Copying file://indexes/only_preds_preds_one_line_per_document_greedy_decode_1_pred/_0.fdm [Content-Type=application/octet-stream]...
Copying file://indexes/only_preds_preds_one_line_per_document_beam_search_1_pred/_0.tvx [Content-Type=application/octet-stream]...
Copy

Total size:

In [94]:
!gsutil du -shc gs://marcospiau-doc2query-aula05

60.46 GiB    gs://marcospiau-doc2query-aula05
60.46 GiB    total
